# NoSQL (MongoDB) (Ejemplo de Replica Set y Sharding)

![Image of MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB-Logo-5c3a7405a85675366beb3a5ec4c032348c390b3f142f5e6dddf1d78e2df5cb5c.png)

Esta hoja muestra cómo acceder a bases de datos MongoDB y también a conectar la salida con Jupyter. Se puede utilizar el *shell* propio de MongoDB en la máquina virtual usando el programa `mongo`. La diferencia es que ese programa espera código Javascript y aquí trabajaremos con Python.

In [ ]:
!pip install --upgrade pymongo

In [ ]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

In [ ]:
%%bash
sudo apt-get update
sudo apt-get install -y mongodb-clients

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [ ]:
%%bash
command='rs.initiate();sleep(1000);rs.status()'
echo "$command" | mongo --host mongo_mongocfg1_1

In [ ]:
%%bash
command='rs.initiate();sleep(1000);rs.status()'
echo "$command" | mongo --host mongo_mongors1_1

In [ ]:
import pymongo
from pymongo import MongoClient

La conexión se inicia con `MongoClient` en el `host` descrito en el fichero `docker-compose.yml` (`mongo`).

In [ ]:
client = MongoClient("mongo_mongors1_1",27017)
client

In [ ]:
client.database_names()

In [ ]:
%%bash
file=../Posts.csv
test -e $file || wget http://neuromancer.inf.um.es:8080/es.stackoverflow/`basename ${file}`.gz -O - 2>/dev/null | gunzip > $file

In [ ]:
db = client.stackoverflow
db = client['stackoverflow']
db

In [ ]:
posts = db.posts
posts

Importación de los ficheros CSV. Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

In [ ]:
%%bash
mongoimport --db stackoverflow --collection posts --drop --type csv \
    --headerline --host=mongo_mongors1_1 --file ../Posts.csv

In [ ]:
posts.count()

El API de colección en Python se puede encontrar aquí: https://api.mongodb.com/python/current/api/pymongo/collection.html. La mayoría de libros y referencias muestran el uso de mongo desde Javascript, ya que el *shell* de MongoDB acepta ese lenguaje. La sintaxis con respecto a Python cambia un poco, y se puede seguir en el enlace anterior.

In [ ]:
post = posts.find_one()
post

In [ ]:
posts.find({"PostTypeId": 2}).explain()

In [ ]:
from bson.code import Code

In [ ]:
map = Code(
'''
function () {
    emit(this.OwnerUserId, 1);
}
''')

In [ ]:
reduce = Code(
'''
function (key, values)
{
    return Array.sum(values);
}
''')

In [ ]:
results = db.posts.map_reduce(map, reduce, "myresults")

In [ ]:
db.collection_names()

In [ ]:
list(results.find())